In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
from sklearn.svm import LinearSVC
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from PIL import Image
import PIL.ImageOps
from collections import defaultdict
from glob import glob
from random import shuffle, seed
import pylab as pl
import pandas as pd
import re
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from multiprocessing import Process, freeze_support

In [2]:
def img_to_array(filename):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    img = list(img.getdata())
    #img = map(list, img)
    img = np.array(img)
    s = img.shape[0] * img.shape[1]
    img_wide = img.reshape(1, s)
    return img_wide[0]

shirt = glob('bag/dataset/150new/100/shirt_resized/*')
collared_tees = glob('bag/dataset/150new/100/collared_tees_resized/*')
normal_tees = glob('bag/dataset/150new/100/normal_tees_resized/*')

process_file = img_to_array

raw_data = [(process_file(filename),'shirt',filename) for filename in shirt] + \
           [(process_file(filename),'collared_tees',filename) for filename in collared_tees] + \
           [(process_file(filename),'normal_tees',filename) for filename in normal_tees]

# randomly order the data
seed(0)
shuffle(raw_data)

In [3]:
data = np.array([cd for (cd,_y,f) in raw_data])
labels = np.array([_y for (cd,_y,f) in raw_data])
y = [(0 if label == 'collared_tees' else( 1 if label=='normal_tees' else 2 )) for label in labels ]

# Scaling the words
stdSlr = StandardScaler().fit(data)
data = stdSlr.transform(data)

from sklearn.cross_validation import train_test_split,cross_val_score,KFold,StratifiedShuffleSplit,StratifiedKFold
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=2)

C:\Users\admin\Anaconda\lib\site-packages\sklearn\utils\validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int32
  "got %s" % (estimator, X.dtype))


In [8]:
'''
c=np.column_stack((data,y))
import pandas as pd
df = pd.DataFrame(c) # A is a numpy 2d array
df.to_csv('dataset.csv',index=False, float_format="%f")
'''

In [8]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=1000, 
                           criterion='entropy',
                           max_depth=7,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           max_features='auto',
                           bootstrap=False, 
                           random_state=2,
                           verbose=0)
                           #n_jobs=-1)
                           #min_density=None, 
                           #compute_importances=None
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred)) # accuracy = 0.86592178

0.672240802676


# 10 fold cross validation

In [5]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
clf=RandomForestClassifier(n_estimators=1000, 
                           criterion='entropy',
                           max_depth=7,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           max_features='auto',
                           bootstrap=False, 
                           random_state=1,
                           verbose=0)

scores = cross_validation.cross_val_score(clf, data, y, cv=8,n_jobs= -1)

In [6]:
scores.mean()

0.53216098446361604

# 5 FOld Cross Validation

In [8]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
clf=RandomForestClassifier(n_estimators=1000, 
                           criterion='entropy',
                           max_depth=7,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           max_features='auto',
                           bootstrap=False, 
                           random_state=1,
                           verbose=0, 
                           min_density=None, 
                           compute_importances=None)

scores = cross_validation.cross_val_score(clf, data, y, cv=2)

In [9]:
scores.mean()

0.55690355219732501

# Hyper parameter tuning

In [5]:
from sklearn.grid_search import GridSearchCV
feature_option=['auto','sqrt',0.5]
depth_option = [0.5,5,7,9,None]
split_option = [1,4,7,9,11]
leaf_options=[1,4,7,9,11]
param_grid = dict(max_features=feature_option,
                  max_depth=depth_option,
                  min_samples_split=split_option,
                  min_samples_leaf =leaf_options)
print param_grid

{'min_samples_split': [1, 4, 7, 9, 11], 'max_features': ['auto', 'sqrt', 0.5], 'max_depth': [0.5, 5, 7, 9, None], 'min_samples_leaf': [1, 4, 7, 9, 11]}


In [6]:
from sklearn.ensemble import RandomForestClassifier
'''
grid_clf = RandomForestClassifier(criterion='gini',
                                  max_depth=7,
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  max_features='auto',
                                  random_state=1)
 '''                                 
grid_clf = RandomForestClassifier(n_estimators= 900,criterion='entropy',random_state=1)

In [ ]:
grid =GridSearchCV(grid_clf,param_grid,cv=5,scoring ='accuracy')
grid.fit(data,y)

In [ ]:
grid.grid_scores_

#Testing arbitrary Image

In [11]:
img = cv2.imread("bag/test/thumbnails/ftest7.jpg")

#img = list(img.getdata())
#img = map(list, img)
img = np.array(img)
s = img.shape[0] * img.shape[1]*img.shape[2]
img_wid = img.reshape(1, s)
img.shape

(100, 100, 3)

In [12]:
img_wid[0]
stdSlr = StandardScaler().fit(img_wid[0])
img_wid1 = stdSlr.transform(img_wid[0])
y_pred = clf.predict(img_wid1)
y_pred

C:\Users\admin\Anaconda\lib\site-packages\sklearn\utils\validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got uint8
  "got %s" % (estimator, X.dtype))


array([0])